In [1]:
import pymysql
import requests

下載分析pm2.5 opendata資料

In [2]:
url="https://data.moenv.gov.tw/api/v2/aqx_p_02?api_key=e8dd42e6-9b8b-43f8-991e-b3dee723a52d&limit=1000&sort=datacreationdate%20desc&format=JSON"
url

'https://data.moenv.gov.tw/api/v2/aqx_p_02?api_key=e8dd42e6-9b8b-43f8-991e-b3dee723a52d&limit=1000&sort=datacreationdate%20desc&format=JSON'

In [3]:
resp=requests.get(url,verify=False)
resp

c:\Users\user\anaconda3\Lib\site-packages\urllib3\connectionpool.py:1097: InsecureRequestWarning: Unverified HTTPS request is being made to host 'data.moenv.gov.tw'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


<Response [200]>

In [4]:
datas=resp.json()['records']
datas

JSONDecodeError: Expecting value: line 1 column 1 (char 0)

In [ ]:
len(datas)

In [ ]:
datas[0]

建立資料表 8/20 1:00:00

In [ ]:
table_str=""" 
create table if not exists pm25(
id int auto_increment primary key,
site varchar(25),
county varchar(50),
pm25 int,
datacreationdate datetime,
itemunit varchar(20),
unique key site_time(site,datacreationdate)
)
"""
print(table_str)

連結資料庫

conn=pymysql.connect(
host="127.0.0.1",
user="root",
password="",
port=3307,
database="demo"
)

In [6]:
conn=pymysql.connect(
host="mysql-c87907a-arthur201010-4fba.j.aivencloud.com",
user="avnadmin",
password="AVNS_HWzrTchr2PZUlSOko5r",
port=11977,
database="defaultdb"
)
cursor= conn.cursor()
cursor

In [ ]:
cursor.execute(table_str)
conn.commit()

寫入一筆資料

In [ ]:
sqlstr="insert into pm25(site,county,pm25,datacreationdate,itemunit)\
      values('{}','{}',{},'{}','{}')"
data=list(datas[0].values())
print(data)
sqlstr.format(data[0],data[1],data[2],data[3],data[4])

In [ ]:
cursor.execute(sqlstr.format(data[0],data[1],data[2],data[3],data[4]))
conn.commit()

### 一次寫入 8/20 1:40:00
- 加上 ignore 8/20 2:00:00

In [ ]:
sqlstr="insert ignore into pm25(site,county,pm25,datacreationdate,itemunit)\
      values(%s,%s,%s,%s,%s)"
values=[list(data.values()) for data in datas if list(data.values())[2]!=""]
#values
cursor.executemany(sqlstr,values)
conn.commit()

In [7]:
conn.close()

In [ ]:
import pandas as pd

In [ ]:
cursor.execute("select * from pm25")
datas=cursor.fetchall()
datas

In [ ]:
df=pd.DataFrame(datas,columns=["id","site","county","pm25","datetime","unit"])
df

In [ ]:
df.groupby("county").get_group("新北市").drop_duplicates()

In [ ]:
len(df)

In [ ]:
df[["site","datetime"]].drop_duplicates()

In [ ]:
print(5+6)